# Here we are preprocessing images from the raw/input, turning them grayscale, resizing(1024 -> 256),

# First, we generate filtered images in-style
### Styles: Gaussian blur, posterize, outline, night_vis. The night_vis filter is made in Figma for test purposes

In [2]:
import os
import cv2
import numpy as np

# =============================
#   PATHS
# =============================
RAW_ROOT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/raw"
PREP_ROOT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed"

BASE_INPUT = os.path.join(RAW_ROOT, "input")

STYLE_FOLDERS_RAW = {
    "blur":       os.path.join(RAW_ROOT, "style/blur"),
    "outline":    os.path.join(RAW_ROOT, "style/outline"),
    "poster":     os.path.join(RAW_ROOT, "style/poster"),
    "night_vis":  os.path.join(RAW_ROOT, "style/night_vis"),
}

STYLE_FOLDERS_PREP = {
    name: os.path.join(PREP_ROOT, f"style/{name}")
    for name in ["blur", "outline", "poster", "night_vis"]
}

PREP_INPUT = os.path.join(PREP_ROOT, "input")


# =============================
#   MAKE DIRECTORIES
# =============================
os.makedirs(PREP_INPUT, exist_ok=True)
for d in STYLE_FOLDERS_PREP.values():
    os.makedirs(d, exist_ok=True)

RESIZE_TO = (256, 256)
VALID_EXT = {".jpg", ".jpeg", ".png", ".bmp", ".tiff"}


# =============================
#   STYLE FILTER FUNCTIONS
# =============================

def strong_gaussian_blur(img):
    """Strong Gaussian blur for training."""
    return cv2.GaussianBlur(img, (45,45), sigmaX=12)


def posterize_super_cartoon(img, levels=4):
    """Very strong cartoon effect."""
    # Bilateral smooth
    smooth = cv2.bilateralFilter(img, d=15, sigmaColor=90, sigmaSpace=90)

    # Posterize
    x = smooth.astype(np.float32) / 255.0
    x = np.floor(x * levels) / levels
    poster = (x * 255).astype(np.uint8)

    # Extract sharp edges
    gray = cv2.cvtColor(poster, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 80, 180)
    edges = cv2.dilate(edges, np.ones((2,2), np.uint8), iterations=1)
    edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    # Invert edges (black lines)
    edges_inv = 255 - edges

    # Apply edges as mask
    toon = cv2.bitwise_and(poster, edges_inv)
    return toon


def outlines_soft(img, levels=6):
    base = posterize_super_cartoon(img, levels)

    smooth = cv2.bilateralFilter(base, 9, 50, 50)

    gray = cv2.cvtColor(base, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 40, 120)
    edges_soft = cv2.GaussianBlur(edges, (5,5), 0)

    edges_rgb = cv2.cvtColor(edges_soft, cv2.COLOR_GRAY2BGR)

    light_edges = 255 - edges_rgb
    light_edges = (light_edges * 0.30).astype(np.uint8) + 180

    out = cv2.addWeighted(smooth, 1.0, light_edges, 0.45, 0)
    return out

def preprocess_image(path):
    img = cv2.imread(path)
    if img is None:
        return None
    img = cv2.resize(img, RESIZE_TO)
    return img


# =============================
#   PROCESS BASE INPUT
# =============================

print("Processing BASE input images...\n")

for fname in sorted(os.listdir(BASE_INPUT)):
    ext = os.path.splitext(fname)[1].lower()
    if ext not in VALID_EXT:
        continue

    src = os.path.join(BASE_INPUT, fname)
    img = preprocess_image(src)
    if img is None:
        continue

    cv2.imwrite(os.path.join(PREP_INPUT, fname), img)

    # AUTO-GENERATED styles
    cv2.imwrite(os.path.join(STYLE_FOLDERS_PREP["blur"], fname),
                strong_gaussian_blur(img))

    cv2.imwrite(os.path.join(STYLE_FOLDERS_PREP["poster"], fname),
                posterize_super_cartoon(img))

    cv2.imwrite(os.path.join(STYLE_FOLDERS_PREP["outline"], fname),
                outlines_soft(img))


    print("Generated auto styles for:", fname)


# =============================
#   PROCESS RAW STYLE FOLDERS
# =============================

print("\nProcessing RAW style folders (just resize → preprocessed)...\n")

for style_name, raw_dir in STYLE_FOLDERS_RAW.items():
    out_dir = STYLE_FOLDERS_PREP[style_name]

    if not os.path.exists(raw_dir):
        print(f"Skipping missing style folder: {raw_dir}")
        continue

    for fname in sorted(os.listdir(raw_dir)):
        ext = os.path.splitext(fname)[1].lower()
        if ext not in VALID_EXT:
            continue

        src = os.path.join(raw_dir, fname)
        img = preprocess_image(src)
        if img is None:
            continue

        cv2.imwrite(os.path.join(out_dir, fname), img)
        print(f"Copied RAW style '{style_name}': {fname}")

print("\nDone! All preprocessing and style generation completed.")


Processing BASE input images...

Generated auto styles for: apple.png
Generated auto styles for: banana.png
Generated auto styles for: butterfly.png
Generated auto styles for: clown.png
Generated auto styles for: frog.png
Generated auto styles for: grape.png
Generated auto styles for: kiwi.png
Generated auto styles for: ladybug.png
Generated auto styles for: lily.png
Generated auto styles for: lion.png
Generated auto styles for: orange.png
Generated auto styles for: owl.png
Generated auto styles for: seahorse.png

Processing RAW style folders (just resize → preprocessed)...

Copied RAW style 'night_vis': apple_night_vis.png
Copied RAW style 'night_vis': banana_night_vis.png
Copied RAW style 'night_vis': butterfly_night_vis.png
Copied RAW style 'night_vis': clon_night_vis.png
Copied RAW style 'night_vis': frog_night_vis.png
Copied RAW style 'night_vis': grape_night_vis.png
Copied RAW style 'night_vis': kiwi_night_vis.png
Copied RAW style 'night_vis': lady_night_vis.png
Copied RAW style 